# This is the script to create the LiDAR projection image and saved as PNG file for own waymo dataset. It is the same procedure of how we process and LiDAR data in CLFT and CLFCN. For benchmarking experiment of PanFormer, it only works with image-like input, thus we create the LiDAR image as standalone file. The pixel value in LiDAR image corresponding to point's 3D coordinates.

In [23]:
import os
import cv2
import pickle
import json, yaml
import numpy as np
from PIL import Image

In [24]:
datapath = '/home/autolab/Data/waymo'

In [25]:
# These functions copied from /utils/lidar_process.py
def open_lidar(lidar_path, w_ratio, h_ratio, lidar_mean, lidar_std):
    mean_lidar = np.array(lidar_mean)
    std_lidar = np.array(lidar_std)

    file = open(lidar_path, 'rb')
    lidar_data = pickle.load(file)
    file.close()

    points3d = lidar_data['3d_points']
    camera_coord = lidar_data['camera_coordinates']

    # select camera front
    mask = camera_coord[:, 0] == 1
    points3d = points3d[mask, :]
    camera_coord = camera_coord[mask, 1:3]

    x_lid = (points3d[:, 1] - mean_lidar[0])/std_lidar[0]
    y_lid = (points3d[:, 2] - mean_lidar[1])/std_lidar[1]
    z_lid = (points3d[:, 0] - mean_lidar[2])/std_lidar[2]

    camera_coord[:, 1] = (camera_coord[:, 1]/h_ratio).astype(int)
    camera_coord[:, 0] = (camera_coord[:, 0]/w_ratio).astype(int)

    points_set = np.stack((x_lid, y_lid, z_lid), axis=1).astype(np.float32)

    return points_set, camera_coord

def get_unresized_lid_img_val(h, w, points_set, camera_coord):
    X = np.zeros((h, w))
    Y = np.zeros((h, w))
    Z = np.zeros((h, w))

    rows = np.floor(camera_coord[:, 1])
    cols = np.floor(camera_coord[:, 0])

    X[(rows.astype(int), cols.astype(int))] = points_set[:, 0]
    Y[(rows.astype(int), cols.astype(int))] = points_set[:, 1]
    Z[(rows.astype(int), cols.astype(int))] = points_set[:, 2]

    # X = TF.to_pil_image(X.astype(np.float32))  # Here no need to convert to tensor becuase we use numpy in this script.
    # Y = TF.to_pil_image(Y.astype(np.float32))
    # Z = TF.to_pil_image(Z.astype(np.float32))

    return X.astype(np.float32), Y.astype(np.float32), Z.astype(np.float32)
    #return X, Y, Z

def lidar_dilation(X, Y, Z):
    kernel = np.ones((3, 3), np.uint8)
    X_dilation = cv2.dilate(np.array(X).astype(np.float32), kernel,
                            iterations=1)
    Y_dilation = cv2.dilate(np.array(Y).astype(np.float32), kernel,
                            iterations=1)
    Z_dilation = cv2.dilate(np.array(Z).astype(np.float32), kernel,
                            iterations=1)

    # X_dilation = TF.to_pil_image(X_dilation.astype(np.float32))
    # Y_dilation = TF.to_pil_image(Y_dilation.astype(np.float32))
    # Z_dilation = TF.to_pil_image(Z_dilation.astype(np.float32))
    return X_dilation.astype(np.float32), Y_dilation.astype(np.float32), Z_dilation.astype(np.float32)
    #return X, Y, Z

In [26]:
train_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/train_all.txt', dtype=str)
valid_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/early_stop_valid.txt', dtype=str)

In [27]:
valid_img_list

array(['labeled/night/rain/camera/segment-10988649507921773627_363_000_383_000_0000000010.png',
       'labeled/day/not_rain/camera/segment-14739149465358076158_4740_000_4760_000_with_camera_labels_0000000156.png',
       'labeled/day/not_rain/camera/segment-3711598698808133144_2060_000_2080_000_with_camera_labels_0000000082.png',
       ...,
       'labeled/day/not_rain/camera/segment-12681651284932598380_3585_280_3605_280_with_camera_labels_0000000062.png',
       'labeled/day/not_rain/camera/segment-5458962501360340931_3140_000_3160_000_with_camera_labels_0000000198.png',
       'labeled/day/rain/camera/segment-15539619898625779290_760_000_780_000_with_camera_labels_0000000124.png'],
      dtype='<U110')

In [ ]:
import torch
import torchvision.transforms.functional as TF
waymo_lidar_mean = [-0.17263354, 0.85321806, 24.5527253] # These are from config.json, specific for own waymo dataset.
waymo_lidar_std = [7.34546552, 1.17227659, 15.83745082]
for imId, paths in enumerate(valid_img_list):
    rgb_path = paths
    lidar_path = os.path.join(datapath + '/' + paths.replace('/camera', '/lidar').replace('.png', '.pkl'))
    
    points_set, camera_coord = open_lidar(lidar_path, w_ratio=4, h_ratio=4, lidar_mean=waymo_lidar_mean, lidar_std=waymo_lidar_std)
    X, Y, Z = get_unresized_lid_img_val(320, 480, points_set, camera_coord)
#X, Y, Z = lidar_dilation(X, Y, Z)

    # X = TF.to_tensor(np.array(X))
    # Y = TF.to_tensor(np.array(Y))
    # Z = TF.to_tensor(np.array(Z))
    #lid_images = torch.cat((X, Y, Z), 0)
    lidar_img = np.dstack((X, Y, Z))
    

    lidar_img_name = rgb_path.split('/')[-1].split('.')[0]
    #imageio.imwrite(os.path.join('/home/autolab/Data/valid_lidar_img/' + lidar_img_name + '.exr'), lidar_img)
    #print(lidar_img)
    cv2.imwrite(os.path.join('/home/autolab/Data/valid_lidar_img/' + lidar_img_name + '.png'), lidar_img)

for imId, paths in enumerate(train_img_list):
    rgb_path = paths
    lidar_path = os.path.join(datapath + '/' + paths.replace('/camera', '/lidar').replace('.png', '.pkl'))
    
    points_set, camera_coord = open_lidar(lidar_path, w_ratio=4, h_ratio=4, lidar_mean=waymo_lidar_mean, lidar_std=waymo_lidar_std)
    X, Y, Z = get_unresized_lid_img_val(320, 480, points_set, camera_coord)
    X, Y, Z = lidar_dilation(X, Y, Z)
    lidar_img = np.dstack((X, Y, Z))

    lidar_img_name = rgb_path.split('/')[-1].split('.')[0]
    
    cv2.imwrite(os.path.join('/home/autolab/Data/train_lidar_img/' + lidar_img_name + '.png'), lidar_img)

In [ ]:
test_day_fair_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/test_day_fair.txt', dtype=str)
test_day_rain_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/test_day_rain.txt', dtype=str)
test_night_fair_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/test_night_fair.txt', dtype=str)
test_night_rain_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/test_night_rain.txt', dtype=str)
#test_night_rain_img_list

In [17]:
for imId, paths in enumerate(test_day_fair_img_list):
    rgb_path = paths
    lidar_path = os.path.join(datapath + '/' + paths.replace('/camera', '/lidar').replace('.png', '.pkl'))
    
    points_set, camera_coord = open_lidar(lidar_path, w_ratio=4, h_ratio=4, lidar_mean=waymo_lidar_mean, lidar_std=waymo_lidar_std)
    X, Y, Z = get_unresized_lid_img_val(320, 480, points_set, camera_coord)
    X, Y, Z = lidar_dilation(X, Y, Z)
    lidar_img = np.dstack((X, Y, Z))

    lidar_img_name = rgb_path.split('/')[-1].split('.')[0]
    
    cv2.imwrite(os.path.join('/home/autolab/Data/test_day_fair_lidar_img/' + lidar_img_name + '.png'), lidar_img)

for imId, paths in enumerate(test_day_rain_img_list):
    rgb_path = paths
    lidar_path = os.path.join(datapath + '/' + paths.replace('/camera', '/lidar').replace('.png', '.pkl'))
    
    points_set, camera_coord = open_lidar(lidar_path, w_ratio=4, h_ratio=4, lidar_mean=waymo_lidar_mean, lidar_std=waymo_lidar_std)
    X, Y, Z = get_unresized_lid_img_val(320, 480, points_set, camera_coord)
    X, Y, Z = lidar_dilation(X, Y, Z)
    lidar_img = np.dstack((X, Y, Z))

    lidar_img_name = rgb_path.split('/')[-1].split('.')[0]
    
    cv2.imwrite(os.path.join('/home/autolab/Data/test_day_rain_lidar_img/' + lidar_img_name + '.png'), lidar_img)

for imId, paths in enumerate(test_night_fair_img_list):
    rgb_path = paths
    lidar_path = os.path.join(datapath + '/' + paths.replace('/camera', '/lidar').replace('.png', '.pkl'))
    
    points_set, camera_coord = open_lidar(lidar_path, w_ratio=4, h_ratio=4, lidar_mean=waymo_lidar_mean, lidar_std=waymo_lidar_std)
    X, Y, Z = get_unresized_lid_img_val(320, 480, points_set, camera_coord)
    X, Y, Z = lidar_dilation(X, Y, Z)
    lidar_img = np.dstack((X, Y, Z))

    lidar_img_name = rgb_path.split('/')[-1].split('.')[0]
    
    cv2.imwrite(os.path.join('/home/autolab/Data/test_night_fair_lidar_img/' + lidar_img_name + '.png'), lidar_img)

for imId, paths in enumerate(test_night_rain_img_list):
    rgb_path = paths
    lidar_path = os.path.join(datapath + '/' + paths.replace('/camera', '/lidar').replace('.png', '.pkl'))
    
    points_set, camera_coord = open_lidar(lidar_path, w_ratio=4, h_ratio=4, lidar_mean=waymo_lidar_mean, lidar_std=waymo_lidar_std)
    X, Y, Z = get_unresized_lid_img_val(320, 480, points_set, camera_coord)
    X, Y, Z = lidar_dilation(X, Y, Z)
    lidar_img = np.dstack((X, Y, Z))

    lidar_img_name = rgb_path.split('/')[-1].split('.')[0]
    
    cv2.imwrite(os.path.join('/home/autolab/Data/test_night_rain_lidar_img/' + lidar_img_name + '.png'), lidar_img)

NameError: name 'get_unresized_lid_img_val' is not defined